# STOCK_PRICE

## Overview
This example function demonstrates the use of the the Alpha Vantage API. You must provide your own free API key from [Alpha Vantage](https://www.alphavantage.co/support/#api-key). You can adapt this example function to use any other stock price API by modifying the URL and response parsing logic.

This example function is provided as-is without any representation or warranty of accuracy.

## Usage
To use the function in Excel:

```excel
=STOCK_PRICE(ticker, api_key)
  - `ticker` (str, required): The stock ticker symbol. Example: "NVDA"
  - `api_key` (str, required): Your Alpha Vantage API key. Example: "your_api_key"

The function returns the current stock price as a float, or an error message as a string if the calculation fails.
```

## Arguments
- `ticker` (str, required): The stock ticker symbol. Example: "NVDA"
- `api_key` (str, required): Your Alpha Vantage API key. Example: "your_api_key"

## Returns
- float: The current stock price, or None if unavailable.
- str: Error message if calculation fails.

## Examples

```excel
=STOCK_PRICE("NVDA", "your_api_key")
=STOCK_PRICE("AAPL", "your_api_key")
```

[Get your free Alpha Vantage API key here.](https://www.alphavantage.co/support/#api-key)


In [ ]:
import requests

def stock_price(ticker, api_key):
    """
    Retrieves the current stock price for a given ticker symbol using the Alpha Vantage API.

    Args:
        ticker (str): The stock ticker symbol.
        api_key (str): Your Alpha Vantage API key.

    Returns:
        float: The current stock price, or None if unavailable.
        str: Error message if calculation fails.

    This example function is provided as-is without any representation or warranty of accuracy.
    """
    if not api_key or api_key == "your_api_key":
        return "Please provide a valid Alpha Vantage API key."
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={ticker}&apikey={api_key}"
    try:
        response = requests.get(url)
        data = response.json()
        if "Global Quote" in data and "05. price" in data["Global Quote"]:
            return float(data["Global Quote"]["05. price"])
        elif "Note" in data:
            return f"API limit reached or invalid API key: {data['Note']}"
        elif "Error Message" in data:
            return f"API error: {data['Error Message']}"
        else:
            return f"API response: {data}"
    except Exception as e:
        return f"Exception occurred: {str(e)}"

In [ ]:
import ipytest
ipytest.autoconfig()
import pytest

demo_cases = [
    ["IBM", "demo"],
    ["AAPL", "your_api_key"]
]

def is_valid_type(val):
    if isinstance(val, (float, bool, str)):
        return True
    if isinstance(val, list):
        return all(isinstance(row, list) and all(isinstance(x, (float, bool, str)) for x in row) for row in val)
    return False

@pytest.mark.parametrize("ticker, api_key", demo_cases)
def test_demo_cases(ticker, api_key):
    result = stock_price(ticker, api_key)
    print(f"Returned value for {ticker}: {result}")
    assert is_valid_type(result), f"Output type is not valid. Got: {type(result)} Value: {result}"

def test_invalid_ticker():
    result = stock_price("INVALIDTICKER", "your_api_key")
    print(f"Returned value for INVALIDTICKER: {result}")
    assert isinstance(result, str)

def test_missing_api_key():
    result = stock_price("AAPL", "")
    print(f"Returned value for missing API key: {result}")
    assert isinstance(result, str)

ipytest.run('-s')

In [ ]:
# Gradio Interface
import gradio as gr

demo = gr.Interface(
    fn=stock_price,
    inputs=[
        gr.Textbox(label="Ticker", value=demo_cases[0][0]),
        gr.Textbox(label="API Key", value=demo_cases[0][1]),
    ],
    outputs=gr.Textbox(label="Stock Price or Message"),
    examples=demo_cases,
    description="Retrieve the current stock price for a given ticker symbol using the Alpha Vantage API. This function is provided as an example only without any representation or warranty of correctness.",
    flagging_mode="never",
    fill_width=True,
)
demo.launch()


In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

api_key = os.getenv('ALPHAVANTAGE_API_KEY', '')
print(f"Loaded API key: {api_key}")
ticker = "NVDA"
if not api_key:
    print("Environment variable ALPHAVANTAGE_API_KEY not set.")
else:
    result = stock_price(ticker, api_key)
    print(f"Stock price for {ticker}: {result}")